# **Latar Belakang**

Dalam era transportasi yang terus berkembang, peningkatan mobilitas penduduk kota dan permintaan layanan transportasi yang lebih efisien telah mendorong pertumbuhan industri transportasi berbasis aplikasi. Salah satu layanan yang populer di New-York adalah TLC atau Taxi and Limousine Commission, yang telah mengalami pertumbuhan pesat dalam beberapa tahun terakhir.

# **Problem Statement**

Industri TLC ini memiliki tantangan penting dalam mengoptimalkan penggunaan armada taksi mereka. Dalam konteks ini, penggunaan optimal merujuk pada pengaturan yang cerdas dan efisien dari armada taksi, dengan tujuan mengurangi waktu tunggu pelanggan, meningkatkan efisiensi operasional, dan pada akhirnya, meningkatkan pengalaman pelanggan. Hal yang bisa dipertimbangkan :

- Waktu padat transaksi
- Hari padat transaksi
- Lokasi penjemputan dan pengantaran transaksi
- Waktu dan jarak tempuh transaksi
- Kemacetan lalu lintas
- Mobilisasi Armada
- Metode Pembayaran

# **Data Understanding**

Untuk kasus ini, kita menggunakan dataset NYC TLC Trip Record tahun 2023. Berikut adalah penjelasan untuk tiap kolom databasenya

- VendorID: Kode yang menunjukkan penyedia LPEP yang menyediakan rekaman. 
  - 1 = Creative Mobile Technologies, LLC. 
  - 2 = VeriFone Inc.
- lpep_pickup_datetime: Tanggal dan waktu saat meteran diaktifkan.
- lpep_dropoff_datetime: Tanggal dan waktu saat meteran dinon-aktifkan.
- Store_and_fwd_flag: catatan perjalanan disimpan di memori kendaraan sebelum dikirim ke vendor, aka “store and forward,” karena kendaraan tidak memiliki koneksi ke server. 
  - Y = store and forward trip 
  - N = not a store and forward trip
- RateCodeID: Kode tarif akhir yang berlaku pada akhir perjalanan.. 
  - 1 = Standard rate 
  - 2 =JFK 
  - 3 =Newark 
  - 4 =Nassau or Westchester 
  - 5 =Negotiated fare 
  - 6 =Group ride
- Passenger_count: Jumlah penumpang.
- Trip_distance: Jarak dalam mil.
- PULocationID: Zona saat meteran diaktifkan.
- DOLocationID: Zona saat meteran dinon-aktifkan.
- Payment_type: Kode numerik yang menandakan pembayaran yang dilakukan oleh penumpang. 
  - 1 = Credit card 
  - 2 = Cash 
  - 3 = No charge 
  - 4 = Dispute 
  - 5 = Unknown 
  - 6 = Voided trip
- Fare_amount: Tarif waktu dan jarak dihitung dengan meteran. Ekstra Lain-lain Ekstra dan biaya tambahan. Saat ini, ini hanya mencakup
  jam sibuk dan biaya malam
- MTA_tax: Pajak MTA sebesar $0,50 secara otomatis berdasarkan tarif meteran yang digunakan.
- Improvement_surcharge: Biaya tambahan perbaikan sebesar $0,30
- Tip_amount: Tip yang diberikan penumpang untuk pembayaran credit card
- Tolls_amount: Total biaya tol jika melewati tol.
- Total_amount: Total biaya yang dibayarkan oleh penumpang.
- Trip_type: Kode yang menandakan penumpang memesan melalui aplikasi atau mencari taxi secara acak di jalan. 
  - 1 = Street-hail 
  - 2 = Dispatch
- eHail fee adalah biaya atau tambahan biaya yang dikenakan kepada pengguna layanan taksi atau transportasi berbasis aplikasi ketika mereka memesan taksi melalui platform elektronik atau aplikasi.
- Congestion surcharge adalah biaya tambahan yang dikenakan kepada pengguna layanan transportasi, seperti taksi atau layanan berbasis aplikasi, ketika mereka menggunakan layanan tersebut dalam kondisi lalu lintas yang padat atau di daerah-daerah dengan kemacetan tinggi.

# **Data Preprocessing**

In [44]:
# import library
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

In [45]:
# load dataset
pd.set_option('display.max_columns', None)
df = pd.read_csv('NYC TLC Trip Record.csv')
df

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2023-01-01 00:26:10,2023-01-01 00:37:11,N,1.0,166,143,1.0,2.58,14.90,1.0,0.5,4.03,0.0,NaN,1.0,24.18,1.0,1.0,2.75
1,2,2023-01-01 00:51:03,2023-01-01 00:57:49,N,1.0,24,43,1.0,1.81,10.70,1.0,0.5,2.64,0.0,NaN,1.0,15.84,1.0,1.0,0.00
2,2,2023-01-01 00:35:12,2023-01-01 00:41:32,N,1.0,223,179,1.0,0.00,7.20,1.0,0.5,1.94,0.0,NaN,1.0,11.64,1.0,1.0,0.00
3,1,2023-01-01 00:13:14,2023-01-01 00:19:03,N,1.0,41,238,1.0,1.30,6.50,0.5,1.5,1.70,0.0,NaN,1.0,10.20,1.0,1.0,0.00
4,1,2023-01-01 00:33:04,2023-01-01 00:39:02,N,1.0,41,74,1.0,1.10,6.00,0.5,1.5,0.00,0.0,NaN,1.0,8.00,1.0,1.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68206,2,2023-01-31 22:29:00,2023-01-31 22:42:00,NaN,NaN,49,62,NaN,4070.82,15.70,0.0,0.0,0.00,0.0,NaN,1.0,16.70,NaN,NaN,NaN
68207,2,2023-01-31 22:40:00,2023-01-31 22:48:00,NaN,NaN,10,205,NaN,2.14,4.41,0.0,0.0,0.00,0.0,NaN,1.0,5.41,NaN,NaN,NaN
68208,2,2023-01-31 23:46:00,2023-02-01 00:02:00,NaN,NaN,66,37,NaN,3.44,16.53,0.0,0.0,3.51,0.0,NaN,1.0,21.04,NaN,NaN,NaN
68209,2,2023-01-31 23:01:00,2023-01-31 23:19:00,NaN,NaN,225,189,NaN,3.03,14.98,0.0,0.0,3.20,0.0,NaN,1.0,19.18,NaN,NaN,NaN


In [46]:
# Cek info tentang masing masing kolom di dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68211 entries, 0 to 68210
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   VendorID               68211 non-null  int64  
 1   lpep_pickup_datetime   68211 non-null  object 
 2   lpep_dropoff_datetime  68211 non-null  object 
 3   store_and_fwd_flag     63887 non-null  object 
 4   RatecodeID             63887 non-null  float64
 5   PULocationID           68211 non-null  int64  
 6   DOLocationID           68211 non-null  int64  
 7   passenger_count        63887 non-null  float64
 8   trip_distance          68211 non-null  float64
 9   fare_amount            68211 non-null  float64
 10  extra                  68211 non-null  float64
 11  mta_tax                68211 non-null  float64
 12  tip_amount             68211 non-null  float64
 13  tolls_amount           68211 non-null  float64
 14  ehail_fee              0 non-null      float64
 15  im

Data awal terdiri dari 68211 baris dan 20 kolom.

Tetapi, data tidak semata-mata bisa digunakan. Perlu ditinjau beberapa hal sebelum data benar-benar digunakan untuk analisis seperti :

- Menyesuaikan tipe kolom
- Handling Data
  - Missing Values
  - Duplikat
  - Anomali

Berdasarkan tipe masing-masing kolom, ada yang perlu diubah yaitu lpep_pickup_datetime dan lpep_dropoff_datetime dari object menjadi datetime

In [47]:
df['lpep_pickup_datetime'] = df['lpep_pickup_datetime'].astype('datetime64[s]')
df['lpep_dropoff_datetime'] = df['lpep_dropoff_datetime'].astype('datetime64[s]')

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68211 entries, 0 to 68210
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype        
---  ------                 --------------  -----        
 0   VendorID               68211 non-null  int64        
 1   lpep_pickup_datetime   68211 non-null  datetime64[s]
 2   lpep_dropoff_datetime  68211 non-null  datetime64[s]
 3   store_and_fwd_flag     63887 non-null  object       
 4   RatecodeID             63887 non-null  float64      
 5   PULocationID           68211 non-null  int64        
 6   DOLocationID           68211 non-null  int64        
 7   passenger_count        63887 non-null  float64      
 8   trip_distance          68211 non-null  float64      
 9   fare_amount            68211 non-null  float64      
 10  extra                  68211 non-null  float64      
 11  mta_tax                68211 non-null  float64      
 12  tip_amount             68211 non-null  float64      
 13  tolls_amount    

Sebelum Handling Data, saya akan melakukan penambahan beberapa kolom untuk mempermudah analisis seperti waktu tempuh, lokasi zona awal dan akhir.

Data lokasi dan zona new york saya ambil dataset berikut https://d37ci6vzurychx.cloudfront.net/misc/taxi+_zone_lookup.csv

In [49]:
# menambahkan kolom waktu tempuh
df['trip_time'] = (df['lpep_dropoff_datetime'] - df['lpep_pickup_datetime']).dt.total_seconds()

# menambahkan kolom hari
df['day'] = df['lpep_pickup_datetime'].dt.strftime('%A')

def map_time_of_day(hour):
    if 6 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 18:
        return 'Afternoon'
    elif 18 <= hour < 21:
        return 'Evening'
    elif 21 <= hour < 24:
        return 'Night'
    else:
        return 'Midnight'

# Apply the mapping function to the 'timestamp' column
df['time_of_day'] = df['lpep_pickup_datetime'].apply(lambda x: map_time_of_day(x.hour))


# menambahkan lokasi awal dan akhir
df_zone = pd.read_csv('taxi+_zone_lookup.csv')

# memasukkan kolom lokasi pickup
df = pd.merge(df,df_zone, left_on = 'PULocationID',right_on = 'LocationID')
df = df.rename(columns={"Borough":"pickup_borough","Zone":"pickup_zone","service_zone":"pickup_service_zone"})

# memasukkan kolom lokasi dropoff
df = pd.merge(df,df_zone, left_on = 'DOLocationID',right_on = 'LocationID')
df = df.rename(columns={"Borough":"dropoff_borough","Zone":"dropoff_zone","service_zone":"dropoff_service_zone"})
df = df.drop(columns=['LocationID_x','LocationID_y','pickup_service_zone','dropoff_service_zone'])

df.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,trip_time,day,time_of_day,pickup_borough,pickup_zone,dropoff_borough,dropoff_zone
0,2,2023-01-01 00:26:10,2023-01-01 00:37:11,N,1.0,166,143,1.0,2.58,14.9,1.00,0.5,4.03,0.0,NaN,1.0,24.18,1.0,1.0,2.75,661.0,Sunday,Midnight,Manhattan,Morningside Heights,Manhattan,Lincoln Square West
1,2,2023-01-02 18:56:58,2023-01-02 19:14:50,N,1.0,166,143,1.0,3.16,19.8,0.00,0.5,0.00,0.0,NaN,1.0,24.05,2.0,1.0,2.75,1072.0,Monday,Evening,Manhattan,Morningside Heights,Manhattan,Lincoln Square West
2,2,2023-01-03 15:28:11,2023-01-03 15:47:47,N,1.0,166,143,1.0,2.55,19.1,0.00,0.5,4.67,0.0,NaN,1.0,28.02,1.0,1.0,2.75,1176.0,Tuesday,Afternoon,Manhattan,Morningside Heights,Manhattan,Lincoln Square West
3,2,2023-01-03 17:37:14,2023-01-03 17:51:35,N,1.0,166,143,1.0,1.96,15.6,2.50,0.5,0.00,0.0,NaN,1.0,22.35,2.0,1.0,2.75,861.0,Tuesday,Afternoon,Manhattan,Morningside Heights,Manhattan,Lincoln Square West
4,1,2023-01-03 19:03:53,2023-01-03 19:17:50,N,1.0,166,143,2.0,2.10,13.5,5.25,1.5,0.00,0.0,NaN,1.0,20.25,2.0,1.0,2.75,837.0,Tuesday,Evening,Manhattan,Morningside Heights,Manhattan,Lincoln Square West


## **Handling Data**

### **Handling Missing Values**

In [50]:
# cek missing values
df.isna().sum()

VendorID                     0
lpep_pickup_datetime         0
lpep_dropoff_datetime        0
store_and_fwd_flag        4324
RatecodeID                4324
PULocationID                 0
DOLocationID                 0
passenger_count           4324
trip_distance                0
fare_amount                  0
extra                        0
mta_tax                      0
tip_amount                   0
tolls_amount                 0
ehail_fee                68211
improvement_surcharge        0
total_amount                 0
payment_type              4324
trip_type                 4334
congestion_surcharge      4324
trip_time                    0
day                          0
time_of_day                  0
pickup_borough               0
pickup_zone                 66
dropoff_borough              0
dropoff_zone               242
dtype: int64

Karena semua data pada ehail_fee bernilai NaN maka kolom ini dapat dihapus karena tidak digunakan juga dalam analisis kali ini

In [51]:
df = df.drop(columns='ehail_fee')
df.isna().sum()

VendorID                    0
lpep_pickup_datetime        0
lpep_dropoff_datetime       0
store_and_fwd_flag       4324
RatecodeID               4324
PULocationID                0
DOLocationID                0
passenger_count          4324
trip_distance               0
fare_amount                 0
extra                       0
mta_tax                     0
tip_amount                  0
tolls_amount                0
improvement_surcharge       0
total_amount                0
payment_type             4324
trip_type                4334
congestion_surcharge     4324
trip_time                   0
day                         0
time_of_day                 0
pickup_borough              0
pickup_zone                66
dropoff_borough             0
dropoff_zone              242
dtype: int64

Kolom store_and_fwd_flag sesuai yang sudah dijelaskan pada data understanding. Pada kasus ini, kita tidak membutuhkannya analisis. Jadi bisa dihapus kolomnya ataupun diisi dengan nilai modus. Kali ini saya akan coba menghapus saja kolomnya

In [52]:
df = df.drop(columns='store_and_fwd_flag')
df.isna().sum()

VendorID                    0
lpep_pickup_datetime        0
lpep_dropoff_datetime       0
RatecodeID               4324
PULocationID                0
DOLocationID                0
passenger_count          4324
trip_distance               0
fare_amount                 0
extra                       0
mta_tax                     0
tip_amount                  0
tolls_amount                0
improvement_surcharge       0
total_amount                0
payment_type             4324
trip_type                4334
congestion_surcharge     4324
trip_time                   0
day                         0
time_of_day                 0
pickup_borough              0
pickup_zone                66
dropoff_borough             0
dropoff_zone              242
dtype: int64

Kolom RatecodeID merupakan angka yang menunjukkan jenis harga yang diberlakukan. Untuk handle missing values kolom ini, bisa dilakukan dengan melihat lokasi pickup dan dropoff.

- Untuk yang pickup atau dropoffnya berada di JFK Airport, maka akan diganti RatecodeIDnya menjadi 2
- Untuk yang pickup atau dropoffnya berada di Newark Airport, maka akan diganti RatecodeIDnya menjadi 3
- Untuk missing values sisanya, akan dianggap standard rate atau RatecodeIDnya 1

In [53]:
df.loc[((df['PULocationID'] == 132) | (df['DOLocationID'] == 132)) & (df['RatecodeID'].isna()), 'RatecodeID'] = 2
df.loc[((df['PULocationID'] == 1) | (df['DOLocationID'] == 1)) & (df['RatecodeID'].isna()), 'RatecodeID'] = 3
df['RatecodeID'] = df['RatecodeID'].fillna(1)
df.isna().sum()

VendorID                    0
lpep_pickup_datetime        0
lpep_dropoff_datetime       0
RatecodeID                  0
PULocationID                0
DOLocationID                0
passenger_count          4324
trip_distance               0
fare_amount                 0
extra                       0
mta_tax                     0
tip_amount                  0
tolls_amount                0
improvement_surcharge       0
total_amount                0
payment_type             4324
trip_type                4334
congestion_surcharge     4324
trip_time                   0
day                         0
time_of_day                 0
pickup_borough              0
pickup_zone                66
dropoff_borough             0
dropoff_zone              242
dtype: int64

Kolom passenger_count tidak terlalu digunakan untuk analisis kali ini, jadi nilai missing values ataupun yang nilainya 0 bisa diisi dengan nilai modus

In [54]:
df['passenger_count'] = df['passenger_count'].replace(0,df['passenger_count'].mode().values[0])
df['passenger_count'] = df['passenger_count'].fillna(df['passenger_count'].mode().values[0])
df.isna().sum()

VendorID                    0
lpep_pickup_datetime        0
lpep_dropoff_datetime       0
RatecodeID                  0
PULocationID                0
DOLocationID                0
passenger_count             0
trip_distance               0
fare_amount                 0
extra                       0
mta_tax                     0
tip_amount                  0
tolls_amount                0
improvement_surcharge       0
total_amount                0
payment_type             4324
trip_type                4334
congestion_surcharge     4324
trip_time                   0
day                         0
time_of_day                 0
pickup_borough              0
pickup_zone                66
dropoff_borough             0
dropoff_zone              242
dtype: int64

Kolom payment_type bisa diisi dengan nomor 6 (VOIDED TRIP) apabila tidak ada trip distance ataupun total amount. Selain itu ada beberapa kondisi lain seperti:

- Apabila ada tip amount maka payment_typenya 1 (CREDIT CARD)
- Apabila tidak ada tip tetapi total amountnya ada, bisa diganti menjadi 5 (UNKNOWN)

In [55]:
df.loc[(df['trip_distance'] == 0) & (df['total_amount'] == 0) & (df['payment_type'].isna()), 'payment_type'] = 6
df.loc[(df['tip_amount'] != 0) & (df['payment_type'].isna()), 'payment_type'] = 1
df['payment_type'] = df['payment_type'].fillna(5)
df.isna().sum()


VendorID                    0
lpep_pickup_datetime        0
lpep_dropoff_datetime       0
RatecodeID                  0
PULocationID                0
DOLocationID                0
passenger_count             0
trip_distance               0
fare_amount                 0
extra                       0
mta_tax                     0
tip_amount                  0
tolls_amount                0
improvement_surcharge       0
total_amount                0
payment_type                0
trip_type                4334
congestion_surcharge     4324
trip_time                   0
day                         0
time_of_day                 0
pickup_borough              0
pickup_zone                66
dropoff_borough             0
dropoff_zone              242
dtype: int64

Kolom trip_type karena tidak terlalu digunakan juga untuk analisis dan tidak ada data yang pasti untuk menentukan apakah street-hail atau dispatch. Maka kita isi dengan nilai modus saja

In [56]:
df['trip_type'] = df['trip_type'].fillna(df['trip_type'].mode().values[0])
df.isna().sum()

VendorID                    0
lpep_pickup_datetime        0
lpep_dropoff_datetime       0
RatecodeID                  0
PULocationID                0
DOLocationID                0
passenger_count             0
trip_distance               0
fare_amount                 0
extra                       0
mta_tax                     0
tip_amount                  0
tolls_amount                0
improvement_surcharge       0
total_amount                0
payment_type                0
trip_type                   0
congestion_surcharge     4324
trip_time                   0
day                         0
time_of_day                 0
pickup_borough              0
pickup_zone                66
dropoff_borough             0
dropoff_zone              242
dtype: int64

Kolom congestion_surcharge merupakan biaya tambahan ketika terjadi kemacetan. Dan itu berpengaruh kepada total amount sedangkan total_amount semua nilainya sudah ada bahkan ketika nilai nilai missing values diatas tadi belum terisi. Maka dari itu kita asumsikan untuk nilai congestion_surcharge pada missing values ini diisi menjadi 0

In [57]:
df['congestion_surcharge'] = df['congestion_surcharge'].fillna(0)
df.isna().sum()

VendorID                   0
lpep_pickup_datetime       0
lpep_dropoff_datetime      0
RatecodeID                 0
PULocationID               0
DOLocationID               0
passenger_count            0
trip_distance              0
fare_amount                0
extra                      0
mta_tax                    0
tip_amount                 0
tolls_amount               0
improvement_surcharge      0
total_amount               0
payment_type               0
trip_type                  0
congestion_surcharge       0
trip_time                  0
day                        0
time_of_day                0
pickup_borough             0
pickup_zone               66
dropoff_borough            0
dropoff_zone             242
dtype: int64

Setelah saya mengambil data dari https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page dan mendownload https://d37ci6vzurychx.cloudfront.net/misc/taxi+_zone_lookup.csv' untuk 2023

Ada 2 ID yang memiliki nilai Unknown dan N/A untuk zonenya. Jadi data tersebut akan di drop untuk melanjutkan analisis berikutnya

In [58]:
df = df[(df['PULocationID'] != 264) & (df['DOLocationID'] != 264) & (df['PULocationID'] != 265) & (df['DOLocationID'] != 265)]
df.isna().sum()

VendorID                 0
lpep_pickup_datetime     0
lpep_dropoff_datetime    0
RatecodeID               0
PULocationID             0
DOLocationID             0
passenger_count          0
trip_distance            0
fare_amount              0
extra                    0
mta_tax                  0
tip_amount               0
tolls_amount             0
improvement_surcharge    0
total_amount             0
payment_type             0
trip_type                0
congestion_surcharge     0
trip_time                0
day                      0
time_of_day              0
pickup_borough           0
pickup_zone              0
dropoff_borough          0
dropoff_zone             0
dtype: int64

### **Handling Duplicate**

In [59]:
# cek duplicate
df.duplicated().sum()

0

Tidak ada data yang duplikat

### **Handling Anomaly**

Ada beberapa kondisi dimana data tersebut anomali. Kita akan coba menelusurinya satu satu per kolom

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 67499 entries, 0 to 68210
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype        
---  ------                 --------------  -----        
 0   VendorID               67499 non-null  int64        
 1   lpep_pickup_datetime   67499 non-null  datetime64[s]
 2   lpep_dropoff_datetime  67499 non-null  datetime64[s]
 3   RatecodeID             67499 non-null  float64      
 4   PULocationID           67499 non-null  int64        
 5   DOLocationID           67499 non-null  int64        
 6   passenger_count        67499 non-null  float64      
 7   trip_distance          67499 non-null  float64      
 8   fare_amount            67499 non-null  float64      
 9   extra                  67499 non-null  float64      
 10  mta_tax                67499 non-null  float64      
 11  tip_amount             67499 non-null  float64      
 12  tolls_amount           67499 non-null  float64      
 13  improvement_surcharge

#### **Trip Distance**

Kota new york memiliki luas 468,9 sq mil, apabila kita asumsikan new york adalah sebuah lingkaran maka keliling dari new york adalah 76,8 mil dimana saya asumsikan keliling tersebut seharusnya adalah jarak terjauh yang bisa ditempuh selama masih berada dalam kawasan new york. Maka dari itu kita buang anomali data ketika trip distancenya kurang dari 0 ataupun lebih dari 155 mil

In [61]:
# Cek terlebih dahulu ada berapa data yang tidak memenuhi kondisi

df[(df['trip_distance'] <= 0) | (df['trip_distance'] > 76.8)]

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,trip_time,day,time_of_day,pickup_borough,pickup_zone,dropoff_borough,dropoff_zone
105,1,2023-01-12 14:05:46,2023-01-12 14:24:09,1.0,41,143,1.0,0.0,14.00,0.0,0.5,1.5,0.00,1.0,19.50,1.0,1.0,0.0,1103.0,Thursday,Afternoon,Manhattan,Central Harlem,Manhattan,Lincoln Square West
229,1,2023-01-12 10:39:21,2023-01-12 11:00:16,1.0,75,143,1.0,0.0,20.04,0.0,0.5,0.0,0.00,1.0,24.04,5.0,1.0,0.0,1255.0,Thursday,Morning,Manhattan,East Harlem South,Manhattan,Lincoln Square West
231,1,2023-01-23 20:34:18,2023-01-23 20:48:26,1.0,75,143,1.0,0.0,16.57,0.0,0.5,0.0,0.00,1.0,20.57,5.0,1.0,0.0,848.0,Monday,Evening,Manhattan,East Harlem South,Manhattan,Lincoln Square West
441,1,2023-01-03 18:42:59,2023-01-03 19:01:57,1.0,231,143,1.0,0.0,23.50,0.0,1.5,0.0,0.00,1.0,25.00,1.0,1.0,0.0,1138.0,Tuesday,Evening,Manhattan,TriBeCa/Civic Center,Manhattan,Lincoln Square West
442,1,2023-01-25 12:29:32,2023-01-25 13:12:47,1.0,165,143,1.0,0.0,31.20,0.0,1.5,0.0,0.00,1.0,32.70,1.0,1.0,0.0,2595.0,Wednesday,Afternoon,Brooklyn,Midwood,Manhattan,Lincoln Square West
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68182,1,2023-01-02 21:02:09,2023-01-02 21:28:46,1.0,216,108,1.0,0.0,39.20,0.0,1.5,0.0,0.00,1.0,40.70,1.0,1.0,0.0,1597.0,Monday,Night,Queens,South Ozone Park,Brooklyn,Gravesend
68189,1,2023-01-08 13:11:22,2023-01-08 13:45:44,1.0,85,108,1.0,0.0,24.50,0.0,1.5,0.0,0.00,1.0,26.00,1.0,1.0,0.0,2062.0,Sunday,Afternoon,Brooklyn,Erasmus,Brooklyn,Gravesend
68195,2,2023-01-19 14:43:16,2023-01-19 14:43:27,5.0,23,23,1.0,0.0,109.25,0.0,0.0,0.0,0.00,1.0,110.25,1.0,2.0,0.0,11.0,Thursday,Afternoon,Staten Island,Bloomfield/Emerson Hill,Staten Island,Bloomfield/Emerson Hill
68202,1,2023-01-13 19:51:38,2023-01-13 20:18:25,1.0,14,44,1.0,0.0,40.20,0.0,1.5,0.0,25.50,1.0,67.20,1.0,1.0,0.0,1607.0,Friday,Evening,Brooklyn,Bay Ridge,Staten Island,Charleston/Tottenville


In [62]:
# TRIP DISTANCE BERNILAI NEGATIF DAN NILAI 0

df = df[(df['trip_distance'] > 0) & (df['trip_distance'] <= 76.8)]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 64470 entries, 0 to 68210
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype        
---  ------                 --------------  -----        
 0   VendorID               64470 non-null  int64        
 1   lpep_pickup_datetime   64470 non-null  datetime64[s]
 2   lpep_dropoff_datetime  64470 non-null  datetime64[s]
 3   RatecodeID             64470 non-null  float64      
 4   PULocationID           64470 non-null  int64        
 5   DOLocationID           64470 non-null  int64        
 6   passenger_count        64470 non-null  float64      
 7   trip_distance          64470 non-null  float64      
 8   fare_amount            64470 non-null  float64      
 9   extra                  64470 non-null  float64      
 10  mta_tax                64470 non-null  float64      
 11  tip_amount             64470 non-null  float64      
 12  tolls_amount           64470 non-null  float64      
 13  improvement_surcharge

#### **Fare Amount**

Untuk biaya taksi seharusnya tidak bisa 0

In [63]:
df[df['fare_amount']<0]

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,trip_time,day,time_of_day,pickup_borough,pickup_zone,dropoff_borough,dropoff_zone
1418,2,2023-01-03 11:30:37,2023-01-03 11:32:23,1.0,166,166,2.0,0.16,-3.70,0.0,-0.5,0.00,0.0,-1.0,-5.20,3.0,1.0,0.0,106.0,Tuesday,Morning,Manhattan,Morningside Heights,Manhattan,Morningside Heights
1443,2,2023-01-05 15:01:39,2023-01-05 15:02:16,1.0,166,166,1.0,0.13,-3.00,0.0,-0.5,0.00,0.0,-1.0,-4.50,4.0,1.0,0.0,37.0,Thursday,Afternoon,Manhattan,Morningside Heights,Manhattan,Morningside Heights
1587,2,2023-01-18 14:41:30,2023-01-18 14:42:00,1.0,166,166,1.0,0.01,-3.00,0.0,-0.5,0.00,0.0,-1.0,-4.50,3.0,1.0,0.0,30.0,Wednesday,Afternoon,Manhattan,Morningside Heights,Manhattan,Morningside Heights
1783,2,2023-01-19 17:48:00,2023-01-19 17:52:00,1.0,166,166,1.0,0.38,-35.01,0.0,0.0,4.86,0.0,1.0,-29.15,1.0,1.0,0.0,240.0,Thursday,Afternoon,Manhattan,Morningside Heights,Manhattan,Morningside Heights
1794,2,2023-01-05 10:01:30,2023-01-05 10:04:04,1.0,24,166,1.0,0.44,-4.00,0.0,-0.5,0.00,0.0,-0.3,-4.80,3.0,1.0,0.0,154.0,Thursday,Morning,Manhattan,Bloomingdale,Manhattan,Morningside Heights
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63252,2,2023-01-26 20:18:19,2023-01-26 20:19:08,1.0,256,256,2.0,0.05,-3.00,-1.0,-0.5,0.00,0.0,-1.0,-5.50,3.0,1.0,0.0,49.0,Thursday,Evening,Brooklyn,Williamsburg (South Side),Brooklyn,Williamsburg (South Side)
63254,2,2023-01-31 22:44:16,2023-01-31 22:45:18,1.0,256,256,1.0,0.15,-3.70,-1.0,-0.5,0.00,0.0,-1.0,-6.20,3.0,1.0,0.0,62.0,Tuesday,Night,Brooklyn,Williamsburg (South Side),Brooklyn,Williamsburg (South Side)
63333,2,2023-01-17 17:50:01,2023-01-17 17:50:39,1.0,55,55,3.0,0.05,-3.00,-2.5,-0.5,0.00,0.0,-1.0,-7.00,3.0,1.0,0.0,38.0,Tuesday,Afternoon,Brooklyn,Coney Island,Brooklyn,Coney Island
63771,2,2023-01-09 09:38:39,2023-01-09 09:46:08,1.0,65,66,1.0,0.88,-8.60,0.0,-0.5,0.00,0.0,-1.0,-10.10,3.0,1.0,0.0,449.0,Monday,Morning,Brooklyn,Downtown Brooklyn/MetroTech,Brooklyn,DUMBO/Vinegar Hill


Berdasarkan hasil diatas, dapat diasumsikan bahwa terjadi salah penginputan terhadap nilai fare_amoount begitupun untuk beberapa kolom lain seperti mta_tax, extra, dan nilai continous lainnya yang berdampak kepada nilai total_amount

Kita bisa memperbaikinya dengan merubah nilai negatif menjadi nilai positif. Tetapi ada beberapa perubahan yang akan terjadi, khususnya pada nilai total_amount.

Nilai total_amount tidak hanya akan berganti dari negatif ke positif, tapi akan tergenerate ulang jumlahnya dari hasil penjumlahan fare_amount dan kolom biaya lainnya.

In [64]:
# nilai mutlak untuk yang bernilai negatif
df['fare_amount'] = df['fare_amount'].abs()
df['extra'] = df['extra'].abs()
df['mta_tax'] = df['mta_tax'].abs()
df['improvement_surcharge'] = df['improvement_surcharge'].abs()
df['congestion_surcharge'] = df['congestion_surcharge'].abs()

# menjumlahkan ulang nilai total_amount
df['total_amount'] = df['fare_amount'] + df['extra'] + df['mta_tax'] + df['improvement_surcharge'] + df['congestion_surcharge']


Setelah itu kita perlu remove yang nilai total_amountnya tetap 0 ataupun fare_amountnya tetap 0

In [65]:
df = df[(df['total_amount'] > 0) | (df['fare_amount'] > 0)]

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 64459 entries, 0 to 68210
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype        
---  ------                 --------------  -----        
 0   VendorID               64459 non-null  int64        
 1   lpep_pickup_datetime   64459 non-null  datetime64[s]
 2   lpep_dropoff_datetime  64459 non-null  datetime64[s]
 3   RatecodeID             64459 non-null  float64      
 4   PULocationID           64459 non-null  int64        
 5   DOLocationID           64459 non-null  int64        
 6   passenger_count        64459 non-null  float64      
 7   trip_distance          64459 non-null  float64      
 8   fare_amount            64459 non-null  float64      
 9   extra                  64459 non-null  float64      
 10  mta_tax                64459 non-null  float64      
 11  tip_amount             64459 non-null  float64      
 12  tolls_amount           64459 non-null  float64      
 13  improvement_surcharge

In [67]:
unique = []
for col in df.columns :
    if col in ['store_and_fwd_flag', 'RatecodeID','passenger_count','payment_type', 'trip_type', 'congestion_surcharge']:
        unique.append( [col,df[col].unique()])

df_unique = pd.DataFrame(columns=['Column','Unique'],
                     data=unique)
df_unique

,Column,Unique
0,RatecodeID,"[1.0, 5.0, 2.0, 3.0, 4.0, 99.0]"
1,passenger_count,"[1.0, 2.0, 3.0, 6.0, 4.0, 5.0, 8.0, 7.0]"
2,payment_type,"[1.0, 2.0, 5.0, 3.0, 4.0]"
3,trip_type,"[1.0, 2.0]"
4,congestion_surcharge,"[2.75, 0.0, 2.5]"


#### **RatecodeID**

Berdasarkan tabel diatas, masih ada sedikit anomali pada RatecodeID dimana sesuai data understanding, hanya ada kode untuk 1-6.

Untuk itu kita bisa cek data tersebut, apabila sedikit bisa kita hapus

In [68]:
df[df['RatecodeID'] == 99]

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,trip_time,day,time_of_day,pickup_borough,pickup_zone,dropoff_borough,dropoff_zone
12818,1,2023-01-19 01:07:59,2023-01-19 01:17:19,99.0,90,68,1.0,0.6,15.2,0.0,1.5,0.0,0.0,1.0,17.7,1.0,1.0,0.0,560.0,Thursday,Midnight,Manhattan,Flatiron,Manhattan,East Chelsea
47739,1,2023-01-02 16:39:05,2023-01-02 17:31:42,99.0,73,82,1.0,3.7,23.5,0.0,1.5,0.0,0.0,1.0,26.0,1.0,1.0,0.0,3157.0,Monday,Afternoon,Queens,East Flushing,Queens,Elmhurst
62145,1,2023-01-15 13:51:42,2023-01-15 14:37:38,99.0,108,181,1.0,6.5,24.2,0.0,1.5,0.0,0.0,1.0,26.7,1.0,1.0,0.0,2756.0,Sunday,Afternoon,Brooklyn,Gravesend,Brooklyn,Park Slope
63383,1,2023-01-01 19:44:13,2023-01-01 19:58:30,99.0,150,55,1.0,3.3,18.2,0.0,1.5,0.0,0.0,1.0,20.7,1.0,1.0,0.0,857.0,Sunday,Evening,Brooklyn,Manhattan Beach,Brooklyn,Coney Island
64541,1,2023-01-05 20:33:45,2023-01-05 22:46:41,99.0,14,123,1.0,6.2,28.2,0.0,1.5,0.0,0.0,1.0,30.7,1.0,1.0,0.0,7976.0,Thursday,Evening,Brooklyn,Bay Ridge,Brooklyn,Homecrest
64552,1,2023-01-15 17:52:52,2023-01-15 22:11:55,99.0,150,123,1.0,1.6,15.2,0.0,1.5,0.0,0.0,1.0,17.7,1.0,1.0,0.0,15543.0,Sunday,Afternoon,Brooklyn,Manhattan Beach,Brooklyn,Homecrest
64554,1,2023-01-21 20:29:58,2023-01-22 02:59:25,99.0,150,123,1.0,1.4,15.2,0.0,1.5,0.0,0.0,1.0,17.7,1.0,1.0,0.0,23367.0,Saturday,Evening,Brooklyn,Manhattan Beach,Brooklyn,Homecrest
65242,1,2023-01-08 11:36:41,2023-01-08 12:14:59,99.0,61,77,1.0,2.7,22.5,0.0,1.5,0.0,0.0,1.0,25.0,1.0,1.0,0.0,2298.0,Sunday,Morning,Brooklyn,Crown Heights North,Brooklyn,East New York/Pennsylvania Avenue
66681,1,2023-01-23 13:45:11,2023-01-23 23:36:42,99.0,244,248,1.0,28.3,50.2,0.0,1.5,0.0,0.0,1.0,52.7,1.0,1.0,0.0,35491.0,Monday,Afternoon,Manhattan,Washington Heights South,Bronx,West Farms/Bronx River


Hanya ada 9 data yang memiliki ratecodeID = 99 , jadi bisa kita hapus datanya

In [69]:
df = df[df['RatecodeID'] != 99]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 64450 entries, 0 to 68210
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype        
---  ------                 --------------  -----        
 0   VendorID               64450 non-null  int64        
 1   lpep_pickup_datetime   64450 non-null  datetime64[s]
 2   lpep_dropoff_datetime  64450 non-null  datetime64[s]
 3   RatecodeID             64450 non-null  float64      
 4   PULocationID           64450 non-null  int64        
 5   DOLocationID           64450 non-null  int64        
 6   passenger_count        64450 non-null  float64      
 7   trip_distance          64450 non-null  float64      
 8   fare_amount            64450 non-null  float64      
 9   extra                  64450 non-null  float64      
 10  mta_tax                64450 non-null  float64      
 11  tip_amount             64450 non-null  float64      
 12  tolls_amount           64450 non-null  float64      
 13  improvement_surcharge

#### **Pickup Datetime**

Sesuai dataset yang didapat, ini adalah trip record dari bulan Januari 2023.

Maka dari itu kita perlu cek apakah ada tanggal pickup yang tidak termasuk ke Januari 2023

In [70]:
df[(df['lpep_pickup_datetime'].dt.month != 1) | (df['lpep_pickup_datetime'].dt.year != 2023)]

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,trip_time,day,time_of_day,pickup_borough,pickup_zone,dropoff_borough,dropoff_zone
41830,2,2022-12-09 14:55:42,2022-12-09 15:33:06,1.0,260,164,1.0,5.48,25.5,0.0,0.5,0.00,6.55,0.3,29.05,2.0,1.0,2.75,2244.0,Friday,Afternoon,Queens,Woodside,Manhattan,Midtown South
48595,2,2022-12-09 14:10:49,2022-12-09 14:22:26,1.0,7,56,1.0,5.37,16.0,0.0,0.5,0.00,0.00,0.3,16.80,2.0,1.0,0.00,697.0,Friday,Afternoon,Queens,Astoria,Queens,Corona
51550,2,2023-02-01 03:10:05,2023-02-01 03:40:35,5.0,82,198,2.0,4.58,28.0,0.0,0.0,5.66,0.00,0.3,28.30,1.0,2.0,0.00,1830.0,Wednesday,Midnight,Queens,Elmhurst,Queens,Ridgewood
67728,2,2009-01-01 20:21:27,2009-01-02 11:07:31,1.0,65,178,1.0,7.72,34.5,0.0,0.5,7.20,0.00,1.0,36.00,1.0,1.0,0.00,53164.0,Thursday,Evening,Brooklyn,Downtown Brooklyn/MetroTech,Brooklyn,Ocean Parkway South


Ada 4 data yang tidak termasuk Januari 2023. Jadi kita bisa hapus datanya untuk mempermudah proses analisis

In [71]:
df = df[(df['lpep_pickup_datetime'].dt.month == 1) & (df['lpep_pickup_datetime'].dt.year == 2023)]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 64446 entries, 0 to 68210
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype        
---  ------                 --------------  -----        
 0   VendorID               64446 non-null  int64        
 1   lpep_pickup_datetime   64446 non-null  datetime64[s]
 2   lpep_dropoff_datetime  64446 non-null  datetime64[s]
 3   RatecodeID             64446 non-null  float64      
 4   PULocationID           64446 non-null  int64        
 5   DOLocationID           64446 non-null  int64        
 6   passenger_count        64446 non-null  float64      
 7   trip_distance          64446 non-null  float64      
 8   fare_amount            64446 non-null  float64      
 9   extra                  64446 non-null  float64      
 10  mta_tax                64446 non-null  float64      
 11  tip_amount             64446 non-null  float64      
 12  tolls_amount           64446 non-null  float64      
 13  improvement_surcharge

#### **Trip Time**

Kolom selanjut yang perlu dicek adalah trip_time, trip_time tidak mungkin bernilai negatif.

Apabila ada nilai negatif, kita bisa lihat ada berapa dan jika sedikit maka bisa dihapus

In [72]:
df.sort_values(by=['trip_time'],ascending = False).head(10)

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,trip_time,day,time_of_day,pickup_borough,pickup_zone,dropoff_borough,dropoff_zone
8292,2,2023-01-18 06:58:14,2023-01-19 06:57:10,1.0,74,41,1.0,1.09,7.9,0.0,0.5,0.00,0.0,1.0,9.40,1.0,1.0,0.00,86336.0,Wednesday,Morning,Manhattan,East Harlem North,Manhattan,Central Harlem
14133,2,2023-01-27 08:56:22,2023-01-28 08:55:17,1.0,43,239,1.0,1.89,11.4,0.0,0.5,3.91,0.0,1.0,15.65,1.0,1.0,2.75,86335.0,Friday,Morning,Manhattan,Central Park,Manhattan,Upper West Side South
4595,2,2023-01-31 09:02:52,2023-02-01 09:01:47,1.0,41,42,1.0,0.87,7.9,0.0,0.5,0.00,0.0,1.0,9.40,1.0,1.0,0.00,86335.0,Tuesday,Morning,Manhattan,Central Harlem,Manhattan,Central Harlem North
29133,2,2023-01-19 08:52:10,2023-01-20 08:51:03,3.0,75,75,1.0,0.04,23.0,0.0,0.0,7.20,0.0,1.0,24.00,1.0,1.0,0.00,86333.0,Thursday,Morning,Manhattan,East Harlem South,Manhattan,East Harlem South
25314,2,2023-01-16 20:09:38,2023-01-17 20:08:24,1.0,42,43,1.0,1.63,10.7,1.0,0.5,0.66,0.0,1.0,13.20,1.0,1.0,0.00,86326.0,Monday,Evening,Manhattan,Central Harlem North,Manhattan,Central Park
54779,2,2023-01-22 02:36:04,2023-01-23 02:34:42,1.0,129,129,1.0,0.42,5.8,1.0,0.5,0.00,0.0,1.0,8.30,2.0,1.0,0.00,86318.0,Sunday,Midnight,Queens,Jackson Heights,Queens,Jackson Heights
10418,2,2023-01-19 16:18:58,2023-01-20 16:17:35,1.0,166,238,1.0,1.20,8.6,2.5,0.5,2.52,0.0,1.0,12.60,1.0,1.0,0.00,86317.0,Thursday,Afternoon,Manhattan,Morningside Heights,Manhattan,Upper West Side North
38734,2,2023-01-03 08:49:50,2023-01-04 08:47:12,1.0,74,141,1.0,3.22,19.1,0.0,0.5,0.00,0.0,1.0,23.35,1.0,1.0,2.75,86242.0,Tuesday,Morning,Manhattan,East Harlem North,Manhattan,Lenox Hill West
13100,2,2023-01-22 13:16:38,2023-01-23 13:13:04,1.0,24,239,1.0,1.99,13.5,0.0,0.5,4.44,0.0,1.0,17.75,1.0,1.0,2.75,86186.0,Sunday,Afternoon,Manhattan,Bloomingdale,Manhattan,Upper West Side South
3764,2,2023-01-20 12:55:21,2023-01-21 12:51:19,1.0,75,229,1.0,2.99,20.5,0.0,0.5,2.40,0.0,0.3,24.05,1.0,1.0,2.75,86158.0,Friday,Afternoon,Manhattan,East Harlem South,Manhattan,Sutton Place/Turtle Bay North


Untuk melihat anomali pada data trip_time ini saya menggunakan asumsi keliling sebesar 76.8 mil, dan kecepatan rata-rata di AS adalah 27 kph atau sekitar 17 mph.

Maka dari itu saya mengasumsikan maksimal berkendara adalah 5 jam.

Saya akan mencoba drop data yang memiliki waktu lebih dari 5 jam dan juga yang kurang dari 5 detik dikarenakan saya mengasumsikan apabila ada yang dibawah 5 detik tandanya terdapat kesalahan dalam pengambilan data ataupun tripnya canceled tapi tidak tercatat

In [73]:
df = df[(df['trip_time']<=18000) & (df['trip_time']>=5)]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 64145 entries, 0 to 68210
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype        
---  ------                 --------------  -----        
 0   VendorID               64145 non-null  int64        
 1   lpep_pickup_datetime   64145 non-null  datetime64[s]
 2   lpep_dropoff_datetime  64145 non-null  datetime64[s]
 3   RatecodeID             64145 non-null  float64      
 4   PULocationID           64145 non-null  int64        
 5   DOLocationID           64145 non-null  int64        
 6   passenger_count        64145 non-null  float64      
 7   trip_distance          64145 non-null  float64      
 8   fare_amount            64145 non-null  float64      
 9   extra                  64145 non-null  float64      
 10  mta_tax                64145 non-null  float64      
 11  tip_amount             64145 non-null  float64      
 12  tolls_amount           64145 non-null  float64      
 13  improvement_surcharge

In [74]:
# sorting berdasarkan tanggal dan jam
df = df.sort_values(by='lpep_pickup_datetime')
df.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,trip_time,day,time_of_day,pickup_borough,pickup_zone,dropoff_borough,dropoff_zone
445,2,2023-01-01 00:01:31,2023-01-01 00:16:02,1.0,166,244,1.0,2.48,16.3,1.0,0.5,3.76,0.0,1.0,18.80,1.0,1.0,0.00,871.0,Sunday,Midnight,Manhattan,Morningside Heights,Manhattan,Washington Heights South
39416,2,2023-01-01 00:04:25,2023-01-01 00:17:50,1.0,255,146,1.0,2.80,16.3,1.0,0.5,2.80,0.0,1.0,18.80,1.0,1.0,0.00,805.0,Sunday,Midnight,Brooklyn,Williamsburg (North Side),Queens,Long Island City/Queens Plaza
45441,2,2023-01-01 00:06:48,2023-01-01 01:08:47,1.0,129,226,2.0,7.83,52.0,1.0,0.5,0.00,0.0,1.0,54.50,2.0,1.0,0.00,3719.0,Sunday,Midnight,Queens,Jackson Heights,Queens,Sunnyside
28441,2,2023-01-01 00:08:05,2023-01-01 00:13:26,1.0,41,75,6.0,1.05,5.5,0.5,0.5,1.36,0.0,0.3,6.80,1.0,1.0,0.00,321.0,Sunday,Midnight,Manhattan,Central Harlem,Manhattan,East Harlem South
19514,2,2023-01-01 00:08:43,2023-01-01 00:17:08,1.0,75,140,1.0,1.99,11.4,1.0,0.5,3.33,0.0,1.0,16.65,1.0,1.0,2.75,505.0,Sunday,Midnight,Manhattan,East Harlem South,Manhattan,Lenox Hill East


Selanjutnya, data yang sudah dibersihkan kita generate ke dalam bentuk CSV untuk dilakukan analisis selanjutnya

In [75]:
df.to_csv('NYC TLC Trip Record Clean.csv',index=False)